In [38]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


In [39]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf
from pmdarima.utils import decomposed_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import acf
from sklearn.preprocessing import MinMaxScaler

# Function to convert age range to a numerical value
def convertir_rango_edad_a_numero(rango_edad):
    return int(rango_edad // 3)

# Function to calculate Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Function to calculate various metrics from model results
def calculo_metricas(results):
    residuals = results.resid
    
    mae = np.mean(np.abs(residuals))
    result = acorr_ljungbox(residuals, period=52)
    mse = np.mean(residuals**2)
    rmse = np.sqrt(mse)
    p_value = round(result.lb_pvalue.mean(), 3)
    
    return mae, mse, rmse, results.aic, results.bic, p_value

# Function to create a summary table from model results
def summary_table(results):
    summary_table = results.summary().tables[1]
    
    # Convert the summary table to a DataFrame
    summary_df = pd.DataFrame(summary_table.data[1:], columns=summary_table.data[0])
    
    # Clean up the DataFrame
    summary_df = summary_df.apply(pd.to_numeric, errors='ignore')
    
    # Rename columns for better access
    summary_df.columns = ['variable', 'coef', 'std_err', 'z', 'P>|z|', '[0.025', '0.975]']
    
    return summary_table


In [41]:
df_f=pd.read_csv("Bases//Base_Limpia_Added.csv")

In [43]:
df_f["provincia"].unique()

df_c0=pd.read_csv("Bases//Base_Casos_Act.csv")


cols_clima=['salud',"densidad_estimada","index obesity","TEMP","HUM","PNM","DD","FF"]


# After a lot of runs the poverty data didnt improve the model, this could is probably because is not temporal data, it only refer to one moment of the census
cols_pobreza=['% de hogares sin acceso a red cloacal','% de población en situación de pobreza crónica','% de población sin obra social ni prepaga'] # Mejor
cols_pobreza=['% de hogares sin acceso a red cloacal','% de población en situación de pobreza crónica']
cols_pobreza=[ 'Nivel de incidencia de pobreza crónica','% de hogares con hacinamiento crítico','% de hogares en vivienda deficitaria','% de hogares sin acceso a red cloacal','% de población en situación de pobreza crónica','% de población sin obra social ni prepaga']

cols_pobreza=['% de hogares sin acceso a red cloacal','% de población en situación de pobreza crónica','% de población sin obra social ni prepaga'] # Mejor
cols_pobreza=[ 'Nivel de incidencia de pobreza crónica','% de hogares con hacinamiento crítico','% de hogares en vivienda deficitaria','% de hogares sin acceso a red cloacal','% de población en situación de pobreza crónica','% de población sin obra social ni prepaga']

cols_pobreza=[]
cols_pred=cols_clima+cols_pobreza

# Select the province or provinces for run the models
l_provincias=[]

l_provincias=["Santa Fe"]

l_provincias=["CABA"]

l_provincias=["Salta","Jujuy"]

l_provincias=["Buenos Aires","CABA"]

array(['Buenos Aires', 'CABA', 'Chaco', 'Cordoba', 'Formosa', 'Jujuy',
       'La Pampa', 'Misiones', 'Salta', 'Santa Fe', 'Tucuman', 'San Luis',
       'Catamarca', 'Entre Rios', 'San Juan', 'Tierra del Fuego',
       'Corrientes', 'Neuquen', 'Chubut', 'Santiago del Estero',
       'Mendoza', 'Santa Cruz', 'Rio Negro', 'La Rioja'], dtype=object)

# ARIMA and SARIMAX Models

In [44]:
def pre_process(l_p, df_c0, df_f, cols_pred):
    """
    Pre-processes tuberculosis incidence and climate data for specific provinces.

    Input:
    - l_p (list): List of provinces to filter data. If empty, processes data for all provinces.
    - df_c0 (DataFrame): DataFrame containing COVID-19 data.
    - df_f (DataFrame): DataFrame containing climate and poverty data.
    - cols_pred (list): List of columns to use as predictors in the analysis.

    Output:
    - df_f2 (DataFrame): Processed DataFrame containing normalized climate and poverty data along with COVID-19 cases.
    - promedio_por_dia (DataFrame): DataFrame with daily averages of predictors.
    - promedio_por_semana (DataFrame): DataFrame with weekly averages of predictors.

    Steps:
    1. Filters data based on the provinces provided in `l_p` or uses all data if `l_p` is empty.
    2. Normalizes columns in `df_f` except 'fecha' using MinMaxScaler.
    3. Computes daily and weekly averages of predictors from `df_f`.
    4. Interpolates missing values in `df_f2`.
    """
    # Step 1: Filter data based on provinces
    if len(l_p) == 0:
        df_cc = df_c0.copy()
        df_f2 = df_f.copy()[cols_pred + ["fecha"]]
    else:
        df_cc = df_c0.loc[df_c0["provincia"].isin(l_p)]
        df_f2 = df_f.loc[df_f["provincia"].isin(l_p)][cols_pred + ["fecha"]]

    # Drop 'provincia' column
    df_cc.drop(["provincia"], axis=1, inplace=True)

    # Step 2: Normalize columns in df_f2 except 'fecha'
    columns_to_normalize = df_f2.columns.difference(['fecha'])
    scaler = MinMaxScaler()
    df_f2 = df_f2.copy()
    df_f2[columns_to_normalize] = scaler.fit_transform(df_f2[columns_to_normalize])
    df_f2['fecha'] = pd.to_datetime(df_f2['fecha'])

    # Step 3: Compute daily and weekly averages of predictors in df_f2
    promedio_por_dia = df_f2.groupby(df_f2['fecha'].dt.to_period('D'))[cols_pred].mean()
    promedio_por_semana = df_f2.groupby(df_f2['fecha'].dt.to_period('W-Mon'))[cols_pred].mean()

    # Step 4: Merge COVID-19 cases with climate and poverty data by week
    df_cc['fecha'] = pd.to_datetime(df_cc['fecha'])
    incidencia_por_semana2 = df_cc.groupby(df_cc['fecha'].dt.to_period('W-Mon'))['casos_corr_2'].sum()
    incidencia_por_semana2 = incidencia_por_semana2.reset_index()

    df_f2 = pd.merge(pd.DataFrame(incidencia_por_semana2, columns=["fecha", "casos_corr_2"]),
                     pd.DataFrame(promedio_por_semana), on="fecha", how="left")
    df_f2 = df_f2.rename(columns={"casos_corr_2": "Casos"})

    # Additional processing: Convert 'fecha' to datetime and interpolate missing values
    df_f2['fecha'] = pd.to_datetime(df_f2['fecha'].astype(str).str.split('/').str[0])
    df_f2 = df_f2.interpolate()

    return df_f2, promedio_por_dia, promedio_por_semana


In [45]:
def easy_models(df_f2, cols_pred):
    """
    Runs and evaluates multiple time series models on COVID-19 data.

    Input:
    - df_f2 (DataFrame): Processed DataFrame containing normalized climate, poverty, and COVID-19 case data.
    - cols_pred (list): List of columns used as predictors in the analysis.

    Output:
    - arima_results (str): Summary table of ARIMA model results.
    - arimax_results (str): Summary table of ARIMAX model results.
    - sarima_results (str): Summary table of SARIMA model results.
    - sarimax_results (str): Summary table of SARIMAX model results.
    - df_metrics (DataFrame): DataFrame containing evaluation metrics (MAE, MSE, RMSE, AIC, BIC, p-value) for each model.
    """
    # ARIMA model without exogenous variables
    model = sm.tsa.ARIMA(df_f2['Casos'], order=(1, 1, 1))
    results = model.fit()
    print("ARIMA results")
    arima_results = summary_table(results)
    arima_metrics = calculo_metricas(results)
    print(arima_metrics)

    # Adjust first fitted value to mean
    results.fittedvalues[0] = results.fittedvalues.mean()

    # ARIMAX model with exogenous variables
    model = sm.tsa.ARIMA(df_f2['Casos'], exog=df_f2[cols_pred], order=(1, 1, 1))
    results = model.fit()
    print("ARIMAX results")
    arimax_results = summary_table(results)
    print(results.summary())
    arimax_metrics = calculo_metricas(results)
    print(arimax_metrics)

    # Auto ARIMA model selection
    print("Auto model results")
    auto_model = auto_arima(df_f2['Casos'], max_p=8, max_d=8, max_q=8)
    print(auto_model.summary())
    p_best = auto_model.order[0]
    d_best = auto_model.order[1]
    q_best = auto_model.order[2]

    # SARIMA model
    order = (p_best, d_best, q_best)
    model = sm.tsa.SARIMAX(df_f2['Casos'], order=order)
    results = model.fit()
    sarima_results = summary_table(results)
    print("SARIMA results")
    results.fittedvalues[0] = results.fittedvalues.mean()
    sarima_metrics = calculo_metricas(results)
    print(sarima_metrics)

    # SARIMAX model with exogenous variables
    model = sm.tsa.SARIMAX(df_f2['Casos'], order=order, exog=df_f2[cols_pred])
    results = model.fit()
    sarimax_results = summary_table(results)
    print("SARIMAX results")
    print(results.summary())
    results.fittedvalues[0] = results.fittedvalues.mean()
    sarimax_metrics = calculo_metricas(results)
    print(sarimax_metrics)

    # Combine metrics into DataFrame for easy comparison
    l_metrics = [arima_metrics, arimax_metrics, sarima_metrics, sarimax_metrics]
    names = ["ARIMA (1,1,1)", "ARIMAX (1,1,1)", f"SARIMA ({p_best},{d_best},{q_best})", f"SARIMAX ({p_best},{d_best},{q_best})"]
    df_metrics = pd.DataFrame(l_metrics, columns=["mae", "mse", "rmse", "AIC", "BIC", "p-value"])
    df_metrics["Model Name"] = names

    return arima_results, arimax_results, sarima_results, sarimax_results, df_metrics


def grid_search(df_f2, best_rmse, model_="SARIMAX"):
    """
    Performs a grid search to find the best SARIMA/SARIMAX model based on RMSE and p-value criteria.

    Input:
    - df_f2 (DataFrame): Processed DataFrame containing normalized climate, poverty, and COVID-19 case data.
    - best_rmse (float): Best RMSE value obtained from initial model evaluation.
    - model_ (str): Type of model to perform grid search ('SARIMAX' or 'ARIMA').

    Output:
    - df_best (DataFrame): DataFrame containing metrics and results of best-performing models from grid search.
    """
    l_metrics = []
    p, d, q = 4, 2, 4  # Define search ranges for p, d, q
    print(f"max p, d, q: {p}, {d}, {q}")

    # Perform grid search over defined parameter ranges
    for p_ in range(p):
        for d_ in range(d):
            for q_ in range(q):
                order = (p_, d_, q_)  # ARIMA/SARIMA order parameters

                try:
                    if model_ == "SARIMAX":
                        model = sm.tsa.SARIMAX(df_f2['Casos'], exog=df_f2[cols_pred], order=order)
                    elif model_ == "ARIMA":
                        model = sm.tsa.ARIMA(df_f2['Casos'], exog=df_f2[cols_pred], order=order)

                    results = model.fit()
                    results.fittedvalues[0] = results.fittedvalues.mean()  # Adjust first fitted value

                    # Calculate metrics
                    mae, mse, rmse, aic, bic, p_value = calculo_metricas(results)

                    # Store results if RMSE is lower than current best and p-value is significant
                    if rmse <= best_rmse and p_value < 0.1:
                        best_results = summary_table(results)
                        print(p_, d_, q_)
                        if p_value < 0.06:
                            print(results.summary())
                        print(mae, mse, rmse, aic, bic, p_value)
                        values = (p_, d_, q_, mae, mse, rmse, aic, bic, p_value, best_results)
                        l_metrics.append(values)

                except:
                    print("Error encountered in model fitting")

    # Create DataFrame from collected metrics and results
    df_best = pd.DataFrame(l_metrics)
    df_best["NAME"] = f"SARIMAX ({df_best[0].astype(str)}, {df_best[1].astype(str)}, {df_best[2].astype(str)})"
    df_best.drop([0, 1, 2], axis=1, inplace=True)
    df_best.columns = ["mae", "mse", "rmse", "AIC", "BIC", "p-value", "results", "Model Name"]

    return df_best


def best_sarimax(df_metrics, model_):
    """
    Identifies the best SARIMA/SARIMAX model based on p-value and RMSE criteria.

    Input:
    - df_metrics (DataFrame): DataFrame containing evaluation metrics for multiple SARIMA/SARIMAX models.
    - model_ (str): Type of model to consider ('SARIMAX' or 'ARIMA').

    Output:
    - df_metrics (DataFrame): Updated DataFrame with details of the best-performing SARIMA/SARIMAX model.
    - df_best (DataFrame): DataFrame containing metrics and results of best-performing models.
    """
    # Display metrics for models with p-value < 0.05
    display(df_metrics[df_metrics["p-value"] < 0.05])

    # Find the best RMSE among models with significant p-value
    best_rmse = df_metrics.loc[df_metrics["p-value"] < 0.05]["rmse"].min()

    # Perform grid search to find best model based on updated RMSE criteria
    df_best = grid_search(df_f2, best_rmse, model_)

    # Identify the best SARIMAX model with p-value < 0.06 and lowest RMSE
    best_sarimax = df_best.loc[df_best["p-value"] < 0.06].sort_values(by="rmse").iloc[0]

    # Append best SARIMAX model details to df_metrics and round values for clarity
    df_metrics.loc[-1] = best_sarimax
    df_metrics = df_metrics.reset_index(drop=True).round(2)

    return df_metrics, df_best


# Example usage:
for l_p in [l_provincias]:
    print(l_p)
    df_f2, promedio_dia, promedio_por_semana = pre_process(l_p, df_c0, df_f, cols_pred)
    arima_results, arimax_results, sarima_results, sarimax_results, df_metrics = easy_models(df_f2, cols_pred)


['Buenos Aires', 'CABA']
ARIMA results
(3.2811015473091207, 17.86447668909888, 4.226638935265098, 4173.663039661218, 4187.442173263646, 0.003)
ARIMAX results
                               SARIMAX Results                                
Dep. Variable:                  Casos   No. Observations:                  731
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -2028.248
Date:                Wed, 26 Jun 2024   AIC                           4078.497
Time:                        19:11:40   BIC                           4129.020
Sample:                             0   HQIC                          4097.989
                                - 731                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
salud                 5.8034      6.85

,mae,mse,rmse,AIC,BIC,p-value,Model Name
0,3.281102,17.864477,4.226639,4173.663040,4187.442173,0.003,"ARIMA (1,1,1)"
1,2.996130,15.265166,3.907066,4078.496877,4129.020367,0.001,"ARIMA(1,1,1)"
2,3.250109,17.510684,4.184577,4161.006245,4179.378423,0.030,"SARIMAX (2, 1, 1)"
3,2.942861,14.686746,3.832329,4053.005875,4108.122409,0.032,"SARIMAX (2, 1, 1)"


In [46]:
best_rmse=df_metrics.loc[df_metrics["p-value"]<0.05]["rmse"].min()
best_rmse

3.832329072205976

In [47]:
def simple_table_to_df(simple_table):
    """
    Convert a simple table structure into a Pandas DataFrame.
    
    Parameters:
    - simple_table (object): A structured data object representing a table.
    
    Returns:
    - df (DataFrame): Pandas DataFrame containing the table data.
    """
    data = simple_table.data[1:]  # Skip the header row, assuming data starts from the second row
    headers = simple_table.data[0]  # Use the first row as headers
    
    # Create DataFrame with extracted data and headers
    df = pd.DataFrame(data, columns=headers)
    
    # Convert all columns to numeric (ignore errors if conversion fails)
    df = df.apply(pd.to_numeric, errors='ignore')
    
    return df

def best_variables(arimax_results, sarimax_results, best_sarimax):
    """
    Consolidates and compares coefficients and statistical significance (P-values) 
    of variables from different model results (ARIMAX, SARIMAX, and best-performing SARIMAX).

    Inputs:
    - arimax_results: Summary table or data structure from ARIMAX model results.
    - sarimax_results: Summary table or data structure from SARIMAX model results.
    - best_sarimax: Detailed results from the best-performing SARIMAX model.

    Outputs:
    - df_final: Pandas DataFrame with variables as rows and columns representing 
      coefficients (coef) and statistical significance (P>|z|) from ARIMAX, SARIMAX, 
      and best-performing SARIMAX models.
    """
    # Extract relevant columns from ARIMAX and SARIMAX results
    df1 = simple_table_to_df(arimax_results)[["", "coef", "P>|z|"]].add_suffix('_ARIMAX')
    df2 = simple_table_to_df(sarimax_results)[["", "coef", "P>|z|"]].add_suffix('_SARIMAX')
    
    # Extract relevant columns from best SARIMAX results
    df3 = simple_table_to_df(best_sarimax["results"])[["", "coef", "P>|z|"]].add_suffix('_SARIMAX_BEST')

    # Rename columns for merging and comparison
    df1 = df1.rename(columns={'_ARIMAX': 'variable'})
    df2 = df2.rename(columns={'_SARIMAX': 'variable'})
    df3 = df3.rename(columns={'_SARIMAX_BEST': 'variable'})

    # Merge DataFrames on 'variable' column
    df_final = pd.merge(df1, df2, on="variable", how="left")
    df_final = pd.merge(df_final, df3, on="variable", how="left")[:-3]  # Remove last 3 rows (assuming they are footer rows)

    # Filter final DataFrame based on significance levels
    df_final = df_final.loc[(df_final["P>|z|_ARIMAX"] < 0.15) | (df_final["P>|z|_SARIMAX"] < 0.05) | (df_final["P>|z|_SARIMAX_BEST"] < 0.15)]

    return df_final



In [48]:
# Filter df_metrics to select rows where the model name starts with "SARIMAX"
# and the p-value is less than or equal to 0.1, indicating statistical significance.
sarimax_true = df_metrics.loc[(df_metrics["Model Name"].str[:7] == "SARIMAX") & (df_metrics["p-value"] <= 0.1)]

# Specify the model type to focus on for further analysis and comparison.
model_best = "SARIMAX"

# Call the function best_sarimax to identify and extract the best-performing SARIMAX model
# from df_metrics based on predefined criteria.
df_metrics, df_best = best_sarimax(df_metrics, model_best)


In [51]:
# Save df_metrics DataFrame to a CSV file with a filename based on l_provincias
df_metrics.to_csv("Resultados\\Tseries_Metricas" + str(l_provincias) + ".csv", index=False)

# Extract the detailed results DataFrame of the best-performing model (last row in df_best) and format it
df_variables = pd.DataFrame(df_best.iloc[-1]["results"])

# Rename the first column to "variable" and set it as the header row
df_variables[0].iloc[0] = "variable"
df_variables.columns = df_variables.iloc[0]

# Remove the first row (header row) to keep only the data rows
df_variables = df_variables[1:]

# Save df_variables DataFrame to a CSV file with a filename based on l_provincias
df_variables.to_csv("Resultados\\Tseries_Variables" + str(l_provincias) + ".csv", index=False)


In [52]:
# Read the variables DataFrame from the CSV file and round numeric values to two decimal places
df_variables = pd.read_csv("Resultados\\Tseries_Variables" + str(l_provincias) + ".csv").round(2)

# Check if there is an 'Unnamed: 0' column in df_variables and drop it if present
if "Unnamed: 0" in df_variables.columns:
    df_variables.drop("Unnamed: 0", axis=1, inplace=True)

# Read the metrics DataFrame from the CSV file and round numeric values to two decimal places
df_metrics = pd.read_csv("Resultados\\Tseries_Metricas" + str(l_provincias) + ".csv").round(2)

# Check if there is an 'Unnamed: 0' column in df_metrics and drop it if present
if "Unnamed: 0" in df_metrics.columns:
    df_metrics.drop("Unnamed: 0", axis=1, inplace=True)

# Convert columns 'AIC' and 'BIC' in df_metrics to integer type
for s in ["AIC", "BIC"]:
    df_metrics[s] = df_metrics[s].astype(int)

# Create a copy of df_metrics for auxiliary purposes
df_aux = df_metrics.copy()

# Drop the 'Model Name' column from df_metrics
df_metrics.drop("Model Name", axis=1, inplace=True)

# Concatenate the 'Model Name' column back to df_metrics after processing
df_metrics = pd.concat([df_aux[["Model Name"]], df_metrics], axis=1)

# Print the list of provinces l_provincias
print(l_provincias)

# Display or return df_metrics for further analysis or visualization
df_metrics


['Buenos Aires', 'CABA']


,Model Name,mae,mse,rmse,AIC,BIC,p-value
0,"ARIMA (1,1,1)",3.28,17.86,4.23,4173,4187,0.00
1,"ARIMA(1,1,1)",3.00,15.27,3.91,4078,4129,0.00
2,"SARIMAX (2, 1, 1)",3.25,17.51,4.18,4161,4179,0.03
3,"SARIMAX (2, 1, 1)",2.94,14.69,3.83,4053,4108,0.03
4,"SARIMAX (3, 1, 2)",2.93,14.60,3.82,4053,4117,0.03


# LSTM Model

In [55]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Filter df_c0 by provinces in l_p
df_cc = df_c0.loc[df_c0["provincia"].isin(l_p)]

# Convert 'fecha' column in df_f2 to datetime
df_f2['fecha'] = pd.to_datetime(df_f2['fecha'])

# Aggregate daily cases into weekly using W-Mon frequency
df_daily = df_f2.groupby(df_f2['fecha'].dt.to_period('W-Mon'))['Casos'].sum().reset_index(name='cases')

# Scale 'cases' using MinMaxScaler to range [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
df_daily['cases_scaled'] = scaler.fit_transform(df_daily['cases'].values.reshape(-1, 1))

# Convert promedio_dia and promedio_por_semana to DataFrames
prom_dia_variables = pd.DataFrame(promedio_dia.to_records())
promedio_por_semana = pd.DataFrame(promedio_por_semana.to_records())

# Convert 'fecha' column to string for merging
prom_dia_variables["fecha"] = prom_dia_variables["fecha"].astype(str)
df_ccc = df_cc[["fecha", "casos_corr_2"]]
df_ccc["fecha"] = df_ccc["fecha"].astype(str)

# Merge prom_dia_variables and df_ccc on 'fecha'
df_data = pd.merge(prom_dia_variables, df_ccc, on=["fecha"], how="left")

# Set 'fecha' column as index and drop it from DataFrame
df_data.index = df_data["fecha"]
df_data = pd.DataFrame(df_data.drop("fecha", axis=1))

# Function to create sequences for LSTM input
def create_sequences(df, seq_length):
    xs = []
    ys = []
    for i in range(len(df) - seq_length):
        x = df.iloc[i:(i + seq_length), 1:].values  # Predictor variables (x1, x2, ..., xn)
        y = df.iloc[i + seq_length, 0]              # Target time series (y)
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Sequence length for LSTM model
seq_length = 10

# Impute missing values in predictor variables
imputer_X = SimpleImputer(strategy='mean')
X_imputed = imputer_X.fit_transform(df_data.drop(columns=["casos_corr_2"]).values)

# Impute NaN values in target variable
imputer_y = SimpleImputer(strategy='mean')
y_imputed = imputer_y.fit_transform(df_data["casos_corr_2"].values.reshape(-1, 1))

# Create sequences for LSTM input
X_seq, y_seq = create_sequences(pd.DataFrame(X_imputed), seq_length)

# Split data into training and test sets
split_ratio = 0.8
split = int(len(X_seq) * split_ratio)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Reshape input dimensions for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Function to define the LSTM model
def rnn_model(X_train):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

    return model, X_train, X_test, y_train, y_test

load = 0

# Create and train the LSTM model
model, X_train, X_test, y_train, y_test = rnn_model(X_train)

# Print model summary
model.summary()

# Evaluate the model using test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.0182 - val_loss: 0.0026
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0023 - val_loss: 8.2783e-04
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0017 - val_loss: 7.4448e-04
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0015 - val_loss: 7.0001e-04
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0015 - val_loss: 7.1834e-04
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0017 - val_loss: 0.0012
Epoch 7/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0015 - val_loss: 7.4272e-04
Epoch 8/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0016 - val_loss: 9.6298e-04
Epoch 9/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 10/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0015 - val_loss: 9.1161e-04
Epoch 11/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0015 - val_loss: 7.7641e-04
Epoch 12/30
18/18 ━━━━━━━━━━━━━

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 10, 100)        │        43,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 10, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 10, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 10, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 10, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 10, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 853,505 (3.26 MB)

 Trainable params: 284,501 (1.09 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 569,004 (2.17 MB)

1/5 ━━━━━━━━━━━━━━━━━━━━ 2s 502ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000223894D63E0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step
Mean Squared Error: 0.0007758029994051187


In [56]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow.keras.backend as K

# Function to calculate log-likelihood
def calculate_log_likelihood(mse, n):
    return -n/2 * np.log(2 * np.pi * mse) - n/2

# Inverse transform predictions and test data to original scale
y_pred_2 = scaler.inverse_transform(y_pred)
y_test_2 = scaler.inverse_transform([y_test])

# Function to count trainable parameters in the model
def count_parameters(model):
    return sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])

# Number of parameters in the model
num_params = count_parameters(model)

# Number of observations
n = len(y_test_2[0])

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test_2[0], y_pred_2[:, 0])

# Calculate log-likelihood
log_likelihood = calculate_log_likelihood(mse, n)

# Calculate Akaike Information Criterion (AIC)
aic = 2 * num_params - 2 * log_likelihood

# Calculate Bayesian Information Criterion (BIC)
bic = np.log(n) * num_params - 2 * log_likelihood

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f'Loglikelihood: {round(log_likelihood, 2)}')
print(f"AIC: {round(aic, 2)}")
print(f"BIC: {round(bic, 2)}")
print(f"MSE: {round(mse, 2)}")
print(f'RMSE: {round(rmse, 3)}')

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test_2[0], y_pred_2[:, 0])
print(f'Mean Absolute Error (MAE): {round(mae, 2)}')


Loglikelihood: -189.06
AIC: 569380.12
BIC: 1416263.85
MSE: 0.79
RMSE: 0.891
Mean Absolute Error (MAE): 0.72


In [57]:
# Select the last row of df_metrics and assign it to df_final
df_final = df_metrics[-1:]

# Append a new row for the LSTM model to df_final
df_final.loc[-1] = ['LSTM Model', round(mae, 2), round(mse, 2), round(rmse, 3), round(aic, 2), round(bic, 2), '']

# Drop the 'p-value' column from df_final (if it exists)
df_final.drop(["p-value"], axis=1, inplace=True)

# Reset the index of df_final
df_final = df_final.reset_index(drop=True)

# Convert 'AIC' column to integer type
df_final["AIC"] = df_final["AIC"].astype(int)

# Round 'rmse' column to 2 decimal places
df_final["rmse"] = df_final["rmse"].round(2)

# Convert 'BIC' column to integer type
df_final["BIC"] = df_final["BIC"].astype(int)

# Display df_final
df_final


,Model Name,mae,mse,rmse,AIC,BIC
0,"SARIMAX (3, 1, 2)",2.93,14.60,3.82,4053,4117
1,LSTM Model,0.72,0.79,0.89,569380,1416263
